# Manejo de Datos en PyTorch

## Librerías

In [1]:
import numpy as np
import pandas as pd

import csv
import gzip
import functools

import torch

from gensim.models import KeyedVectors
from gensim.parsing import preprocessing

from torch.utils.data import Dataset, DataLoader, IterableDataset

## La clase `Dataset`

La clase abstracta [`torch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) es la clase base para construir un dataset de *PyTorch*. Cualquier dataset personalizado debe heredar de dicha clase e implementar los siguientes métodos:

- `__len__`: Para que `len(dataset)` devuelva el tamaño del conjunto de datos.
- `__getitem__`: Para soportar indexado de manera que `dataset[i]` devuelva el elemento `i`. Es común que en ciertos casos se utilice este método para levantar el dato real (e.g. una imagen) mientras que lo que se guarde en el dataset sea sólo una referencia a dicho dato (e.g. un path a la imagen). De esta manera se evita cargar muchas imágenes en memoria, haciendo que sea menos demandante a nivel RAM.

In [2]:
class IMDBReviewsDataset(Dataset):
    def __init__(self, path, transform=None):
        self.dataset = pd.read_csv(path)
        self.transform = transform

    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, item):
        if torch.is_tensor(item):
            item = item.tolist() # Deal with list of items instead of tensor.

        item = {
            'data': self.dataset.iloc[item]['review'],
            'target': self.dataset.iloc[item]['sentiment']
        }

        if self.transform:
            item = self.transform(item)
        
        return item

dataset = IMDBReviewsDataset('data/imdb_reviews.csv.gz')

print(f'Dataset loaded with {len(dataset)} elements')
print(f'Sample element...\n{dataset[0]}')

Dataset loaded with 50000 elements
Sample element...
{'data': "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br />

## Transformaciones

El ejemplo anterior nos muestra el uso básico, pero claramente no podemos pasarle eso a una red neuronal, ya que no puede manejar texto. Es para eso que tenemos que hacer algún tipo de transformación sobre los atributos (en este caso el único atributo es el texto).

### Normalización

En particular, como vemos en el caso anterior, el texto no está normalizado, donde parte de las transformaciones pueden incluir realizar algún tipo de normalización. Para eso hagamos uso de [`gensim`](https://radimrehurek.com/gensim/index.html), en particular utilizaremos el módulo [`preprocessing`](https://radimrehurek.com/gensim/parsing/preprocessing.html#module-gensim.parsing.preprocessing) que se encargará de hacer varias normalizaciones por defecto.

In [3]:
class TextPreprocess:
    def __init__(self, filters=None):
        if filters:
            self.filters = filters
        else:
            # Predefined filters...
            self.filters = [
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]

    def _preprocess_string(self, string):
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _encode_target(self, target):
        return 1 if target == 'positive' else 0

    def __call__(self, item):
        if isinstance(item['data'], str):
            # String
            data = self._preprocess_string(item['data'])
        else:
            # Iterable
            data = [self._preprocess_string(d) for d in item['data']]

        if isinstance(item['target'], str):
            # String
            target = self._encode_target(item['target'])
        else:
            # Iterable
            target = [self._encode_target(t) for t in item['target']]

        return {
            'data': data,
            'target': target
        }

preprocess = TextPreprocess()
# Let's see the results
preprocess(dataset[0])

{'data': ['reviewers',
  'mentioned',
  'watching',
  'episode',
  'hooked',
  'right',
  'exactly',
  'happened',
  'thing',
  'struck',
  'brutality',
  'unflinching',
  'scenes',
  'violence',
  'set',
  'right',
  'word',
  'trust',
  'faint',
  'hearted',
  'timid',
  'pulls',
  'punches',
  'regards',
  'drugs',
  'sex',
  'violence',
  'hardcore',
  'classic',
  'use',
  'word',
  'called',
  'nickname',
  'given',
  'oswald',
  'maximum',
  'security',
  'state',
  'penitentary',
  'focuses',
  'mainly',
  'emerald',
  'city',
  'experimental',
  'section',
  'prison',
  'cells',
  'glass',
  'fronts',
  'face',
  'inwards',
  'privacy',
  'high',
  'agenda',
  'city',
  'home',
  'aryans',
  'muslims',
  'gangstas',
  'latinos',
  'christians',
  'italians',
  'irish',
  'scuffles',
  'death',
  'stares',
  'dodgy',
  'dealings',
  'shady',
  'agreements',
  'far',
  'away',
  'main',
  'appeal',
  'fact',
  'goes',
  'shows',
  'wouldn',
  'dare',
  'forget',
  'pretty',
  'p

### Conversión a vectores

Es posible continuar con la conversión del texto, a una representación por vectores. Si bien hay muchas posibilidades (siendo la *bolsa de palabras* una de las más utilizadas), en general para *Deep Learning* se prefieren representaciones utilizando vectores continuos, obtenidos por algún método del estilo de **Word2vec**, **Glove** o **FastText**. Para este caso utilizaremos las representaciones de *Glove* de dimensión 50 que se dejaron para descargar en el [Notebook 0](0_Set_Up.ipynb).

In [4]:
class VectorizeText:
    def __init__(self, glove_vectors_path):
        self.glove_model = KeyedVectors.load_word2vec_format('data/glove.6B.50d.txt', binary=False, no_header=True)
        self.unknown_vector = np.random.randn(self.glove_model.vector_size) # Random vector for unknown words.

    def _get_vector(self, word):
        if word in self.glove_model:
            # We know the embedding!
            return self.glove_model[word]
        else:
            # Unknown word!
            return self.unknown_vector

    def _get_vectors(self, sentence):
        return np.vstack([self._get_vector(word) for word in sentence])

    def __call__(self, item):
        review = []
        if isinstance(item['data'][0], str):
            # String
            review = self._get_vectors(item['data'])
        else:
            # Iterable
            review = [self._get_vectors(d) for d in item['data']]

        return {
            'data': review,
            'target': item['target']
        }

vectorizer = VectorizeText('data/glove.6B.50d.txt.gz')
# Let's see the results
vectorizer(preprocess(dataset[0]))

{'data': array([[-0.18105   , -0.79229999, -0.097616  , ...,  1.42859995,
         -0.032471  ,  0.47235999],
        [ 0.69395   ,  0.69261003, -0.21608   , ...,  0.2247    ,
         -0.23197   ,  0.0062523 ],
        [-0.0049087 ,  0.12611   ,  0.14056   , ..., -0.58464003,
         -0.31830999,  0.31564   ],
        ...,
        [ 0.25435999, -0.44304001, -0.12524   , ...,  0.73352998,
          0.026198  ,  0.30408001],
        [-0.058468  ,  0.019087  ,  0.089056  , ..., -0.28176001,
          0.045137  , -0.18802001],
        [ 0.14443   ,  0.39103001, -0.93454999, ..., -0.71325999,
         -0.54575998,  0.13952   ]]),
 'target': 1}

### Combinación de vectores

Si bien ahora estamos con una versión de los atributos que podría pasar por una red neuronal, hay un problema. Las distintas reseñas tienen diferentes largos, y como el algoritmo se entrena en lotes (*mini-batches*), estas deberían tener el mismo largo. Hay varias maneras de lidiar con esto, cada una con sus ventajas y desventajas. Dado que por ahora solo vimos *perceptrón multicapa* (**MLP**), que espera algo de tamaño fijo, una opción sencilla puede ser la de promediar el tamaño de los vectores de palabras.

In [5]:
class WordVectorsAverage:
    def __call__(self, item):
        if item['data'][0].ndim == 2:
            data = np.vstack([np.mean(d, axis=0) for d in item['data']])
        else:
            data = np.mean(item['data'], axis=0)

        return {
            'data': data,
            'target': item['target']
        }

vector_average = WordVectorsAverage()
# Let's see the results
vector_average(vectorizer(preprocess(dataset[0])))

{'data': array([ 0.10228207,  0.01455134, -0.1121626 , -0.27668046,  0.27284423,
         0.06560054, -0.05191655, -0.04326527, -0.15702042,  0.19189558,
        -0.21519975, -0.1007045 , -0.13937947, -0.00704294,  0.28695519,
         0.02225536,  0.02181543, -0.01945881, -0.12781747, -0.25174534,
        -0.06460028,  0.30616061,  0.14370052,  0.19194541,  0.09540124,
        -1.08034224, -0.36215775,  0.17903577,  0.42573122, -0.18066451,
         2.04714924,  0.15924272, -0.05178646, -0.40460137, -0.07660328,
         0.10285706, -0.06545683, -0.16149122, -0.20091396, -0.24677175,
        -0.13023138,  0.07413312,  0.04403049,  0.2685901 ,  0.14540364,
         0.00686889, -0.00328018, -0.13658369, -0.05233869, -0.06472035]),
 'target': 1}

### Conversión de vectores a tensores

En el paso final, debemos convertir nuestros datos de arreglos de `numpy` a tensores de *PyTorch*.

In [6]:
class ToTensor:
    def __call__(self, item):
        """
        This espects a single array.
        """
        return {'data': torch.from_numpy(item['data']), 'target': torch.tensor(item['target'])}

to_tensor = ToTensor()
# Let's see the results
to_tensor(vector_average(vectorizer(preprocess(dataset[0]))))

{'data': tensor([ 0.1023,  0.0146, -0.1122, -0.2767,  0.2728,  0.0656, -0.0519, -0.0433,
         -0.1570,  0.1919, -0.2152, -0.1007, -0.1394, -0.0070,  0.2870,  0.0223,
          0.0218, -0.0195, -0.1278, -0.2517, -0.0646,  0.3062,  0.1437,  0.1919,
          0.0954, -1.0803, -0.3622,  0.1790,  0.4257, -0.1807,  2.0471,  0.1592,
         -0.0518, -0.4046, -0.0766,  0.1029, -0.0655, -0.1615, -0.2009, -0.2468,
         -0.1302,  0.0741,  0.0440,  0.2686,  0.1454,  0.0069, -0.0033, -0.1366,
         -0.0523, -0.0647], dtype=torch.float64),
 'target': tensor(1)}

### Componiendo las transformaciones

Para evitar tener que llamar a todas las funciones de transformación que queremos aplicar, hacemos uso del parámetro `transform` que definimos en nuestro `Dataset` (con un poco de ayuda de `functools`).

In [7]:
def compose(*functions):
    return functools.reduce(lambda f, g: lambda x: g(f(x)), functions, lambda x: x)

dataset = IMDBReviewsDataset(
    'data/imdb_reviews.csv.gz',
    transform=compose(preprocess, vectorizer, vector_average, to_tensor)
)

print(f'Dataset loaded with {len(dataset)} elements')
print(f'Sample element...\n{dataset[0]}')

Dataset loaded with 50000 elements
Sample element...
{'data': tensor([ 0.1023,  0.0146, -0.1122, -0.2767,  0.2728,  0.0656, -0.0519, -0.0433,
        -0.1570,  0.1919, -0.2152, -0.1007, -0.1394, -0.0070,  0.2870,  0.0223,
         0.0218, -0.0195, -0.1278, -0.2517, -0.0646,  0.3062,  0.1437,  0.1919,
         0.0954, -1.0803, -0.3622,  0.1790,  0.4257, -0.1807,  2.0471,  0.1592,
        -0.0518, -0.4046, -0.0766,  0.1029, -0.0655, -0.1615, -0.2009, -0.2468,
        -0.1302,  0.0741,  0.0440,  0.2686,  0.1454,  0.0069, -0.0033, -0.1366,
        -0.0523, -0.0647], dtype=torch.float64), 'target': tensor(1)}


### Iterando el conjunto de datos

Ya tenemos nuestro conjunto de datos definitivo con sus respectivas transformaciones. ¿Para qué nos sirve esto? Una opción es simplemente iterar en el conjunto de datos de a un elemento por vez. Esto es sencillo, simplemente se hace a través de un `for`.

In [8]:
for idx, sample in enumerate(dataset):
    print(sample['data'])
    print(sample['target'])
    print('=' * 50)

    if idx == 3:
        # We don't need so many examples...
        break

tensor([ 0.1023,  0.0146, -0.1122, -0.2767,  0.2728,  0.0656, -0.0519, -0.0433,
        -0.1570,  0.1919, -0.2152, -0.1007, -0.1394, -0.0070,  0.2870,  0.0223,
         0.0218, -0.0195, -0.1278, -0.2517, -0.0646,  0.3062,  0.1437,  0.1919,
         0.0954, -1.0803, -0.3622,  0.1790,  0.4257, -0.1807,  2.0471,  0.1592,
        -0.0518, -0.4046, -0.0766,  0.1029, -0.0655, -0.1615, -0.2009, -0.2468,
        -0.1302,  0.0741,  0.0440,  0.2686,  0.1454,  0.0069, -0.0033, -0.1366,
        -0.0523, -0.0647], dtype=torch.float64)
tensor(1)
tensor([ 0.0767,  0.1736, -0.3908, -0.1270,  0.3595,  0.1924, -0.2518, -0.1332,
        -0.1555,  0.3660, -0.0880,  0.2526, -0.1314,  0.1429,  0.2969, -0.0450,
         0.0940,  0.0859, -0.1270, -0.3729,  0.0754,  0.3559,  0.0260,  0.0445,
         0.3602, -0.7753, -0.5883,  0.1104,  0.3951, -0.1837,  2.1237, -0.0937,
         0.0923, -0.4264,  0.0245,  0.3073, -0.0416,  0.2610, -0.2185, -0.3416,
         0.1346,  0.1324, -0.1528,  0.0578, -0.0143,  0.0782, 

## La clase `Dataloader`

El problema con iterar de a un elemento por vez es que estamos limitados al querer entrenar un modelo. Para empezar, los modelos de *Deep Learning* suelen ser más eficientes si se entrenan utilizando algún tipo de entrenamiento por *mini-batches*. Además, hay otras cuestiones como mezclar los elementos (*shuffling*) o cargar datos en paralelo vía distintos *multiprocess workers*. La clase [`torch.utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) precisamente se encarga de hacer eso por nosotros:

In [9]:
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=2)

for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched['data'].size(), sample_batched['target'].size())

    if i_batch == 3:
        # We don't need so many examples...
        print(sample_batched['data'])
        print(sample_batched['target'])
        break

0 torch.Size([4, 50]) torch.Size([4])
1 torch.Size([4, 50]) torch.Size([4])
2 torch.Size([4, 50]) torch.Size([4])
3 torch.Size([4, 50]) torch.Size([4])
tensor([[ 3.0416e-02,  1.4680e-01,  4.8470e-02, -2.3265e-01,  4.3905e-01,
          1.0492e-01, -4.6799e-01, -1.1880e-01, -2.5818e-01,  2.0054e-01,
         -1.8987e-01,  2.6144e-01, -1.7447e-01,  1.4546e-01,  7.0066e-01,
          1.7029e-01,  1.3352e-01,  1.6263e-01, -3.0031e-01, -3.3626e-01,
          1.6313e-02,  3.6978e-01,  2.9381e-01,  2.5543e-01,  4.1670e-01,
         -1.2412e+00, -5.8719e-01,  1.1016e-01,  3.5761e-01, -3.9062e-01,
          2.3470e+00,  2.7216e-01,  4.3461e-02,  2.5046e-02,  2.4115e-02,
          5.8554e-02, -2.5833e-02, -8.1999e-03,  3.2247e-02, -4.1884e-01,
         -1.3350e-01,  2.9274e-01, -2.3388e-02,  7.7495e-02,  8.9617e-02,
          1.2941e-01,  2.0797e-02, -2.7418e-01, -7.5235e-02,  2.6937e-01],
        [-6.2870e-02,  1.4123e-01, -2.9085e-01, -2.9693e-01,  2.9440e-01,
          3.6957e-01, -1.9028e-01

## La clase `IterableDataset`

El método preferido para trabajar con conjuntos de datos en *PyTorch* es `torch.utils.data.Dataset`. En general, hacer uso inteligente del método `__getitem__` es fundamental. Usándolo para cargar imágenes a medida que sean solicitadas y no al instanciar el dataset, es la mejor manera de trabajar con un conjunto de datos. En particular, de esta manera es mucho más fácil hacer *shuffling* de los datos y demás tareas. No obstante, esto no siempre es posible, muchas veces el conjunto de datos es demasiado grande para levantarlo en memoria (aunque sólo levantemos referencias). Para esos casos, *PyTorch* ofrece la clase [`torch.utils.data.IterableDataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset), donde el único método que es requerido implementar es `__iter__`.

In [10]:
class MeLiChallengeDataset(IterableDataset):
    def __init__(self, path, transform=None):
        self.dataset_path = path
        self.transform = transform

    def __iter__(self):
        """
        Since I have to work on a dataset that I CLEARLY don't have,
        I had to forcibly enter my own code to reproduce this cell...
        """
        df = pd.read_csv(self.dataset_path)

        for index, row in df.iterrows():
            item = {'data': row['title'], 'target': row['category']}

            if self.transform:
                yield self.transform(item)
            else:
                yield item

dataset = MeLiChallengeDataset('data/dataset.csv')

dataloader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=0)
dataiterable = iter(dataloader)
print(f'Sample batch...\n{dataiterable.next()}')

Sample batch...
{'data': ['Galoneira Semi Industrial', 'Máquina De Coser Brother Industrial', 'Teclado Casio Wk-240 76 Teclas Profissional Standard', 'Heladera Gafa 380 Impecable Urgente'], 'target': ['SEWING_MACHINES', 'SEWING_MACHINES', 'MUSICAL_KEYBOARDS', 'REFRIGERATORS']}
